# Inference about your images

In [ ]:
!pip install tf_slim
!pip install tensorflow-io
!pip install tf-models-official==2.8.0
!pip install avro-python3 
!pip install tf_slim==1.1.0
!pip install lvis
#!pip install tensorflow_io==0.23.1
!pip install keras==2.7.0
!pip install opencv-python-headless==4.5.2.52

#import tensorflow_io as tfio
import matplotlib.pyplot as plt
import pandas as pd
import os
import io
import tensorflow as tf




In [ ]:
%cd /content
!git clone --quiet https://github.com/tensorflow/models.git
%cd /content/models/
#!git checkout 58d19c67e1d30d905dd5c6e5092348658fed80af
!apt-get update && apt-get install -y -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -q Cython contextlib2 pillow lxml matplotlib
!pip install -q pycocotools
%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'
!python object_detection/builders/model_builder_test.py

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile("/content/fine_tuned_model.zip", 'r')
zip_ref.extractall("/content/fine_tuned_model")
zip_ref.close()

# generate inference about your model like visual form

In [ ]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
import tensorflow as tf
import numpy as np

PATH_TO_MODEL_DIR = "path of your train model"
PATH_TO_SAVE_MODEL = PATH_TO_MODEL_DIR + "/saved_model"

detect_fn = tf.saved_model.load(PATH_TO_SAVE_MODEL)

In [ ]:
label_map_pbtxt_fname = "path of your labelMap file"
category_index = label_map_util.create_category_index_from_labelmap(label_map_pbtxt_fname)

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

image_path = "path of your image test"

image_test = Image.open(image_path)

#image_np = np.array(image_test.resize((1024,1024)))

input_tensor = tf.convert_to_tensor(image_np)
input_tensor = input_tensor[tf.newaxis, ...]
detections = detect_fn(input_tensor)

In [ ]:
num_detections = int(detections.pop('num_detections'))

detections = {key: value[0,: num_detections].numpy() for key, value in detections.items()} 

detections['num_detections'] = num_detections

detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
    image_np_with_detections,
    detections['detection_boxes'],
    detections['detection_classes'],
    detections['detection_scores'],
    category_index,
    max_boxes_to_draw=500,
    min_score_thresh=0.4,
    use_normalized_coordinates= True
)


cv2_imshow(image_np_with_detections)


threshold = 0.4
detection_count = 0

for i, (y_min, x_min, y_max, x_max) in enumerate(detections['detection_boxes']):
  # validates if score has a acceptable value and if its class match with expected class
  if detections['detection_scores'][i] > threshold and ("cultivo" == None or category_index[detections['detection_classes'][i]]['name'] in "cultivo"):
    detection_count += 1

print(detection_count)